In [2]:
import json
import os
import pandas as pd
from datetime import datetime
import pickle 
from matplotlib import pyplot as plt
from collections import Counter
import numpy as np
import torch
import logging
from tqdm import tqdm
import ast

from utils.tree_utils import *
from utils.utils import *
from dataset import *
from model import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
import math

In [4]:
out_dir = '../data/'
data_dir = '../data/'
journalist = 'JiayangFan'
batch_size = 4

In [5]:
dp = []
rel = []
journal_sort = pd.read_csv((os.path.join(out_dir, f'{journalist}/{journalist}_conv_labels.csv')))
ids = []
for item in list(journal_sort['conversation_id']):
    if item not in ids:
        ids.append(item)
id_pair = {}
id_conv = {}
for idx in ids:
    id_pair[idx], id_conv[idx] = create_conversation_list(journal_sort[journal_sort['conversation_id']==idx], idx)
id_data, uid, data, label = create_data(journal_sort, ids)
prob = pkl.load(open(os.path.join(data_dir, f'{journalist}/{journalist}_edgeprob.pkl'), 'rb'))


In [7]:
link_label = []
for idx in ids:
    temp = []
    conv = journal_sort[journal_sort['conversation_id']==idx]
    for index, row in conv.iterrows():
        if row['reference_id'] == idx:
            temp.append(1)
        else:
            temp.append(0)
    link_label.append(temp)


In [17]:
## LR
acc = []
f1 = []
prec = []
recall = []
for i in range(len(prob)):
    X = prob[i]
    y = link_label[i]
    if len(X) <= 1:
        continue
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    if len(set(y_train)) == 1:
        continue
    # Initialize the logistic regression model
    model = LogisticRegression()
    # Fit the model to the training data
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    acc.append(accuracy_score(y_test, predictions))
    prec.append(precision_score(y_test, predictions))
    recall.append(recall_score(y_test, predictions))
    f1.append(f1_score(y_test, predictions))


print(np.mean(acc), np.mean(recall), np.mean(f1))



/u/yian3/.conda/envs/py39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/u/yian3/.conda/envs/py39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/u/yian3/.conda/envs/py39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/u/yian3/.conda/envs/py39/lib/python3.9/

0.7123129103994068 0.7397491422675474 0.6982606731558519


In [16]:
## DMM
import numpy as np
from scipy.stats import dirichlet

num_strategies = 8
num_iterations = 10
# Define the alpha parameter for the Dirichlet distribution
alpha = np.array([1.0] * num_strategies)  # Adjust based on your model's needs



In [50]:
def compute_likelihood(l_aij_sk, theta_k):
    """
    Compute the overall likelihood of responses.
    
    Parameters:
    - l_aij_sk: Array of likelihoods l(a_{ij}|s_k) for each strategy and user pair.
    - theta_k: The current probability distribution over strategies.
    
    Returns:
    - Total likelihood of the responses.
    """
    total_likelihood = torch.sum(torch.Tensor(l_aij_sk) * theta_k, axis=1)  # Assuming axis 0 is strategies
    return torch.sum(total_likelihood)  # Sum over all user pairs


In [46]:
## flatten
l_aij_sk = []
for item in prob:
    for x in item:
        l_aij_sk.append(x)

flat_label = []
for item in link_label:
    for x in item:
        flat_label.append(x)

In [53]:
train_sk = []
test_sk = []
test_label = []
for i in range(len(flat_label)):
    if i < 1000 and flat_label[i] == 1:
        train_sk.append(l_aij_sk[i])
    else:
        test_sk.append(l_aij_sk[i])
        test_label.append(flat_label[i])

In [54]:
theta_k = torch.tensor(dirichlet.rvs(alpha, size=1)[0], requires_grad=True)

# Define optimizer
optimizer = optim.Adam([theta_k], lr=0.01)

# Optimization loop
for _ in tqdm(range(num_iterations)):
    optimizer.zero_grad()
    
    # Compute likelihood (example, adjust according to your actual computation)
    likelihood = compute_likelihood(train_sk, theta_k)
    
    # Negative likelihood for minimization
    loss = -likelihood
    loss.backward()
    optimizer.step()

100%|██████████| 10/10 [00:01<00:00,  6.85it/s]


In [64]:
pred = []
for i in range(len(test_sk)):
    result = np.sum(np.array(test_sk[i]) * theta_k.detach().numpy())
    if  result > 0.7:
        pred.append(1)
    else:
        pred.append(0)

In [66]:
acc = []
f1 = []
prec = []
recall = []
for i in range(len(pred)):
    if test_label[i]== pred[i]:
        c += 1
print(c / len(pred))

0.650987751478096
